In [2]:
# Installing Folium for making a simple map w'll use folium.Map function and set location and zoom_start for appearing the map.
pip install folium

import folium 
import numpy as np 
import pandas as pd 

In [3]:
# Reading the gamma data in the dataframes.

df_eneregy_rating=pd.read_csv('data/energy_rating_data.csv')
df_ecad_data = pd.read_csv('data/ecad_20221216.csv')

In [ ]:
# Joining the ecad_data with energy rating data.

df_ecad_energy_rating = pd.merge(df_ecad_data, df_eneregy_rating, on=['ecad_id'])
df_ecad_energy_rating.head()

In [5]:
# Color coding the energy rating.

colors={'A1':'darkgreen' , 
'A2':'darkgreen'  ,        
'A3':'darkgreen'   ,        
'B1':'green',   
'B2':'green',    
'B3':'green',       
'C1':'lightgreen',   
'C2':'lightgreen', 
'C3':'lightgreen',
'D1':'beige',
'D2':'oranges',
'E1':'oranges',
'E2':'lightred',
'F' :'red',
'G' :'darkred',
       }

In [6]:
# Creating the popup labels for energy rating data for building id to display on selection of the icon marker in the map.

def popup_html(row):
    i = row
    Building_ID=df_ecad_energy_rating['building_id'].iloc[i] 
    Address= df_ecad_energy_rating['address_line_1'].iloc[i]
    Ber_Rating = df_ecad_energy_rating['ber_rating'].iloc[i]
    Ber_Rating_Kwh = df_ecad_energy_rating['ber_rating_kwh'].iloc[i]
    Co2_Emission = df_ecad_energy_rating['co2_emission'].iloc[i]

    left_col_color = "#3e95b5"
    right_col_color = "#f2f9ff"
    
    html = """
<!DOCTYPE html>
<html>
<center><h4 style="margin-bottom:5"; width="200px">{}</h4>""".format(Building_ID) + """</center>
<center> <table style="height: 126px; width: 305px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Address </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">"""+ Address + """</td>
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Ber_Rating </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(Ber_Rating) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Ber_Rating_Kwh </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(Ber_Rating_Kwh) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Co2_Emission </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(Co2_Emission) + """
</tr>
</tbody>
</table></center>
</html>
"""
    return html

In [ ]:
# Displaying the Ber Rating, Ber Rating kwh and CO2 emission by address and building_id.

location = df_ecad_energy_rating['etrs89_lat'].mean(), df_ecad_energy_rating['etrs89_long'].mean()
m_cluster = folium.Map(location=location,zoom_start=12)
from folium import plugins
from folium.plugins import FloatImage
cluster=plugins.MarkerCluster().add_to(m_cluster)
image_file = 'data/fd_gamma_data-20221216/ber.PNG'
FloatImage(image_file, bottom=0, left=86).add_to(cluster)    
m = folium.Map(location=location,zoom_start=6)
for i in range(0,len(df_ecad_energy_rating)): 
    html = popup_html(i)
    popup = folium.Popup(folium.Html(html, script=True), max_width=500)
    folium.Marker([df_ecad_energy_rating['etrs89_lat'].iloc[i],df_ecad_energy_rating['etrs89_long'].iloc[i]],
                  popup=popup,icon=folium.Icon(color=colors[df_ecad_energy_rating['ber_rating'].iloc[i]],icon='building', prefix='fa')).add_to(cluster)

m_cluster

m_cluster.save("ecad_energy_rating.html")